In [19]:
#loading needed modules from biopython
from Bio import Entrez, Seq, SeqIO
import numpy as np

#load nucleotide sequences from NCBI 
def get_seq(db, key, rettype="gb"):
    Entrez.email = "A.N.Other@example.com"
    handle = Entrez.efetch(db=db, id=key, rettype=rettype)
    return SeqIO.read(handle, "gb")

#[welche daten werden geladen?]
#hvr-1
nea_1a = get_seq("nucleotide", "AF011222")
nea_1b = get_seq("nucleotide", "AF282971")

#[welche daten werden geladen?]
#hvr-2
nea_2a = get_seq("nucleotide", "AF142095")
nea_2b = get_seq("nucleotide", "AF282972")

In [21]:
n = 206 #known length
with open("d_loop.fasta", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        print(record.id)

Andalusia(AF382011).txt
Aust(AF346963).txt
Aust(AF346964).txt
Aust(AF346965).txt
Aust(AY289051).txt
Aust(AY289052).txt
Aust(AY289053).txt
Aust(AY289054).txt
Aust(AY289055).txt
Aust(AY289056).txt
Aust(AY289057).txt
Aust(AY289058).txt
Aust(AY289059).txt
Aust(AY289060).txt
Aust(AY289061).txt
Aust(AY289062).txt
Aust(AY289063).txt
Aust(AY289064).txt
Aust(AY289065).txt
Aust(AY289066).txt
Aust(AY289067).txt
Bamileke(AF346967).txt
Berber(AF381989).txt
Berber(AF381990).txt
Biaka(AF346968).txt
Biaka(AF346969).txt
Buriat(AF346970).txt
Buriat(AY519484).txt
Canary(AF381982).txt
Canary(AF382009).txt
Canary(AF382010).txt
Caucasian(AY195745).txt
Caucasian(AY195746).txt
Caucasian(AY195747).txt
Caucasian(AY195750).txt
Caucasian(AY195751).txt
Caucasian(AY195752).txt
Caucasian(AY195754).txt
Caucasian(AY195758).txt
Caucasian(AY195764).txt
Caucasian(AY195765).txt
Caucasian(AY195767).txt
Caucasian(AY195768).txt
Caucasian(AY195769).txt
Caucasian(AY195774).txt
Caucasian(AY195775).txt
Caucasian(AY195778).txt
Ca

In [10]:
nea_1 = nea_1a.seq + nea_1b.seq

In [ ]:
score = np.zeros[]